### Pedalcast: Predicting daily bike usage for each station - Modeling & Feature Engineering Notebook

#### Imports

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import joblib
import holidays

In [40]:
daily_counts = pd.read_csv('data/daily_counts_with_weather.csv')
daily_counts['date'] = pd.to_datetime(daily_counts['date'])

### Feature Engineering: `create_features(df)`

This function generates key features to support station-level bike demand forecasting. It takes a DataFrame with daily trip counts and returns a new DataFrame with additional time-based and lag features, including:

- **Calendar Features**:  
  `dayofweek`, `is_weekend`, `month`, `quarter`, `year`, `dayofyear`, `dayofmonth`, and `weekofyear`  
  These capture patterns related to the day, seasonality, and temporal cycles.

- **Season Label**:  
  A simplified `season` column is derived from the month and one-hot encoded.

- **Lag Features** (per station):  
  - `lag_1`: Trip count from the previous day  
  - `lag_7`: Trip count from the same day the previous week  
  - `rolling_7`: 7-day rolling average of trip count (excluding the current day)  

These features allow the model to learn from both recent usage trends and structured temporal patterns.

In [41]:
def create_features(df):
    out = df.copy()
    # Monday(0) - Sunday (6)
    out['dayofweek'] = out['date'].dt.dayofweek
    out['is_weekend'] = out['dayofweek'] >= 5
    out['quarter'] = out['date'].dt.quarter 
    out['year'] = out['date'].dt.year 
    out['month'] = out['date'].dt.month 
    out['dayofyear'] = out['date'].dt.dayofyear 
    out['dayofmonth'] = out['date'].dt.day 
    out['weekofyear'] = out['date'].dt.isocalendar().week 
   
    def assign_season(month):
        if month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        elif month in [9, 10, 11]:
            return 'Fall'
        else:
            return 'Winter'

    out['season'] = out['month'].apply(assign_season)
    
    out = out.sort_values(['start_station_id', 'date'])

    out['lag_1'] = out.groupby('start_station_id')['trip_count'].shift(1)

    out['lag_7'] = out.groupby('start_station_id')['trip_count'].shift(7)
    
    out['rolling_7'] = (
        out.groupby('start_station_id')['trip_count']
           .shift(1)  # Exclude today
           .rolling(window=7)
           .mean()
           .reset_index(level=0, drop=True)
    )
    
    us_holidays = holidays.UnitedStates()

    out['is_holiday'] = out['date'].isin(us_holidays)
    
    return out

In [42]:
daily_counts_wft = create_features(daily_counts)
daily_counts_wft.head()

,start_station_id,start_station_name,date,trip_count,temp,humidity,precip,snow,windspeed,visibility,...,year,month,dayofyear,dayofmonth,weekofyear,season,lag_1,lag_7,rolling_7,is_holiday
0,A32000,Fan Pier,2023-07-01,30,19.3,93.5,0.000,0.0,20.3,4.8,...,2023,7,182,1,26,Summer,NaN,NaN,NaN,False
1,A32000,Fan Pier,2023-07-02,20,20.9,89.9,8.278,0.0,28.9,7.6,...,2023,7,183,2,26,Summer,30.0,NaN,NaN,False
2,A32000,Fan Pier,2023-07-03,28,20.4,89.4,14.249,0.0,18.0,10.4,...,2023,7,184,3,27,Summer,20.0,NaN,NaN,False
3,A32000,Fan Pier,2023-07-04,35,20.1,94.4,12.163,0.0,16.4,12.0,...,2023,7,185,4,27,Summer,28.0,NaN,NaN,False
4,A32000,Fan Pier,2023-07-05,41,23.9,78.4,0.000,0.0,17.8,13.3,...,2023,7,186,5,27,Summer,35.0,NaN,NaN,False


### Drop Rows with Missing Lag Features

Lag features like `lag_1`, `lag_7`, and `rolling_7` require prior data to be computed. As a result, the earliest days for each station (where lag values are undefined) contain missing values.
So, we drop them to ensure all training data has complete feature sets, which also preserves model quality by removing rows that lack historical context.

In [43]:
# drop some rows for lags
daily_counts_wft.isna().sum()[daily_counts_wft.isna().sum()!=0]

lag_1         554
lag_7        3877
rolling_7    3877
dtype: int64

In [44]:
# drop values with missing lags
daily_counts_wft = daily_counts_wft.dropna(subset=['lag_1', 'lag_7', 'rolling_7'])

#### Train/Test Split for Forecasting

We separate the dataset into features (`X`) and target (`y`), where `trip_count` is the value we want to predict.

Then, we split the data into:

- **Training data** includes all dates **before June 2025**
- **Test data** includes dates in **June 2025**, which we will use to evaluate model performance on unseen future data

Later, we’ll use **TimeSeriesSplit** for cross-validation on the training data during hyperparameter tuning, to ensure the model is validated on temporally consistent folds.

In [45]:
X = daily_counts_wft.drop(['trip_count'],axis=1,errors='ignore')
y = daily_counts_wft['trip_count']

X_train = X[X['date'] < '2025-06-01']
X_test = X[X['date'] >= '2025-06-01']
y_train = y.loc[X_train.index]
y_test = y.loc[X_test.index]

We drop columns `start_station_name` and `date` since they are non-numeric and redundant (already encoded). We needed them before to do certain operations like in the previous cell.

In [46]:
drop_cols = ['start_station_name', 'date']
X_train = X_train.drop(columns=drop_cols, errors='ignore')
X_test = X_test.drop(columns=drop_cols, errors='ignore')

#### One Hot Encode Station IDs and Season - Nominal Qualitative Columns

Since `start_station_id` and `season` is a string, we use `LabelEncoder` to convert it into a numeric format suitable for XGBoost.

#### Modeling Pipeline with One-Hot Encoding and XGBoost

We define a scikit-learn `Pipeline` that performs both preprocessing and model training in a single, reusable workflow:

- **`ColumnTransformer` with `OneHotEncoder`**:
  - Applies one-hot encoding to the categorical columns `start_station_id` and `season`

- **`XGBRegressor`**:
  - A tree-based regression model from the XGBoost library
  - Trained on the encoded and passed-through features

In [47]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(
        handle_unknown='ignore', drop='first'), ['start_station_id','season'])
    ],
    remainder='passthrough'  
)

xgb_pipeline = Pipeline(steps=[
    ('encoder', preprocessor),
    ('model', XGBRegressor(random_state=42))
])

#### Hyperparameter Tuning with GridSearchCV

We perform a grid search over a predefined set of XGBoost hyperparameters to find the combination that minimizes RMSE.

- **Parameters tuned**: `max_depth`, `learning_rate`, `n_estimators`, `subsample`, and `colsample_bytree`
- **Cross-validation**: `TimeSeriesSplit` ensures temporally consistent folds
- **Scoring**: Negative RMSE is used as the optimization metric

The best model configuration is selected based on cross-validated RMSE.

**Note**: Other parameter combinations were also tested, but this setup consistently produced the best performance.

In [48]:
param_grid = {
    "model__max_depth": [3, 5, 7],
    "model__learning_rate": [0.05, 0.1],
    "model__n_estimators": [100, 300],
    "model__subsample": [0.8],
    "model__colsample_bytree": [0.8],
}

tscv = TimeSeriesSplit(n_splits=5)

xgb_grid_search = GridSearchCV(
    xgb_pipeline,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",  # for RMSE
    cv=tscv,
    verbose=2,
    n_jobs=-1
)

xgb_grid_search.fit(X_train, y_train)

print(f"Best RMSE: {-xgb_grid_search.best_score_:.2f}")
print("Best params:", xgb_grid_search.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   0.7s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.0s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.3s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   1.0s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   1.3s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   1.2s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   2.0s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   2.2s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zer

[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   1.8s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   2.8s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   1.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   2.3s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   3.7s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zer

[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   3.0s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   1.4s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   4.6s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   2.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   2.1s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   4.3s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   2.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   0.6s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   0.8s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   3.5s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   5.3s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   3.3s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zer

[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   4.2s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.2s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   6.4s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   1.2s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.5s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   4.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=100, model__subsample=0.8; total time=   1.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   0.7s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   2.0s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   1.2s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   5.9s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   1.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   2.8s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zer

[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   2.1s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   3.5s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   1.8s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   7.0s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=100, model__subsample=0.8; total time=   2.4s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zer

[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   1.1s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   2.6s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=3, model__n_estimators=300, model__subsample=0.8; total time=   4.0s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   1.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.05, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   8.2s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   2.3s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   3.6s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   2.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=100, model__subsample=0.8; total time=   3.2s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   2.7s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   4.5s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=5, model__n_estimators=300, model__subsample=0.8; total time=   4.8s
[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   3.2s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   3.7s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   3.9s


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__colsample_bytree=0.8, model__learning_rate=0.1, model__max_depth=7, model__n_estimators=300, model__subsample=0.8; total time=   4.5s
Best RMSE: 10.13
Best params: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__n_estimators': 300, 'model__subsample': 0.8}


#### Final Model Evaluation on Test Set

We use the best model from `GridSearchCV` to predict bike demand on the June 2025 test set.

- **RMSE (Root Mean Squared Error)** measures the average prediction error in trip counts.
- **R² Score** indicates how much variance in the target is explained by the model.

These metrics give a final assessment of how well the model generalizes to unseen future data.

In [49]:
y_pred = xgb_grid_search.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Test RMSE: {rmse:.2f}")

r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

Test RMSE: 9.24
R² Score: 0.9233


/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [50]:
xgb_grid_search.best_params_

{'model__colsample_bytree': 0.8,
 'model__learning_rate': 0.05,
 'model__max_depth': 7,
 'model__n_estimators': 300,
 'model__subsample': 0.8}

This indicates strong predictive performance.

#### Most Important Features for the Model

**XGBoost’s Built-in Feature Importances**  
   - `feature_importances_` returns the relative importance of each feature based on how much it contributes to reducing error (gain) across all trees.
   - A horizontal bar plot visualizes which features the model relied on most globally.

feature  importance
565                rolling_7    0.419362
563                    lag_1    0.216932
564                    lag_7    0.057072
351  start_station_id_M32006    0.024609
550                   precip    0.016515
3    start_station_id_A32004    0.011455
556               is_weekend    0.010662
378  start_station_id_M32037    0.009363
554               cloudcover    0.007892
555                dayofweek    0.007104
553               visibility    0.006678
8    start_station_id_A32010    0.006591
560                dayofyear    0.005559
460  start_station_id_S32002    0.005071
562               weekofyear    0.005001
360  start_station_id_M32018    0.004638
354  start_station_id_M32011    0.004565
231  start_station_id_D32017    0.004241
71   start_station_id_B32016    0.004103
258  start_station_id_D32045    0.004019


In [51]:
# Step 1: Get encoded feature names from OneHotEncoder
ohe = xgb_grid_search.best_estimator_.named_steps['encoder'].named_transformers_['cat']
encoded_feature_names = ohe.get_feature_names_out(['start_station_id', 'season'])

# Step 2: Combine encoded + passthrough column names
# These are columns not one-hot encoded
passthrough_cols = [col for col in X_train.columns if col not in ['start_station_id', 'season']]
feature_names = list(encoded_feature_names) + passthrough_cols

# Step 3: Get model importances (same length as feature_names)
model = xgb_grid_search.best_estimator_.named_steps['model']
importances = model.feature_importances_

# Step 4: Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Step 6: Plot
plt.figure(figsize=(10, 8))
plt.barh(importance_df['feature'][:20], importance_df['importance'][:20])
plt.xlabel("Feature Importance (Gain)")
plt.title("XGBoost Feature Importances")
plt.tight_layout()
plt.show()

# Print top 20 features
print(importance_df.head(20))

In [52]:
importance_df

,feature,importance
565,rolling_7,0.419362
563,lag_1,0.216932
564,lag_7,0.057072
351,start_station_id_M32006,0.024609
550,precip,0.016515
...,...,...
244,start_station_id_D32031,0.000000
242,start_station_id_D32029,0.000000
241,start_station_id_D32028,0.000000
238,start_station_id_D32025,0.000000


#### Interpretation of XGBoost Feature Importances

The model's internal feature importance ranking confirms that recent activity is the strongest predictor of daily bike demand:



- **`rolling_7`** (7-day rolling average) is by far the most important feature, contributing about 60% of the total gain (the average improvement in the model’s loss function when a feature is used to split a node). This suggests strong weekly patterns in usage.
- **`lag_1`** and **`lag_7`** also rank highly, capturing short- and long-term temporal dependencies.
- **Station-specific effects** are learned through one-hot encoding of `start_station_id` — several station IDs (e.g., `M32006`, `M32037`, `A32004`) appear among the top 20 features. This indicates that the model learns meaningful differences in demand behavior across locations, but without overfitting to them.
- **Weather features** like `precip`, `visibility`, and `cloudcover` contribute a bit but less than lag features.
- **Calendar features** (`dayofweek`, `is_weekend`) also contribute a bit, indicating that while time of week and seasonality affect demand, they’re secondary to usage history.


Overall, the model relies primarily on recent trip patterns, with station-specific effects, weather, and calendar signals providing useful context.

### Final Model Training on All Data

After evaluating the model's performance and interpreting feature importances, we retrain the final XGBoost pipeline on the **entire dataset** — including both the training and test sets.

This ensures that the model has access to the full set of historical patterns before being saved or deployed with the best hyperparameters from gridsearch.

In [53]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

final_pipeline = Pipeline(steps=[
    ('encoder', preprocessor),
    ('model', XGBRegressor(**xgb_grid_search.best_params_, random_state=42))
])

final_pipeline.fit(X_full, y_full)

/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:57:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "model__colsample_bytree", "model__learning_rate", "model__max_depth", "model__n_estimators", "model__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['start_station_id',
                                                   'season'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              en...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              model__colsample_bytree=0.8,
                              model__learning_rate=0.05, model__max_depth=7,
                              model__n_estimators=300, model__subsample=0.8,
                              monotone_constraints=None, ...))])

### Saving and Loading the Final Model

After training, we save the entire modeling pipeline using `joblib`. This includes:
- Some of the preprocessing steps (`OneHotEncoder`)
- The trained `XGBRegressor` model

#### Loading the model later:

To reuse the model in a new notebook and predict:

```python
import joblib
loaded_model = joblib.load('xgb_final_model.pkl')
loaded_model.predict(X_new)

In [54]:
joblib.dump(final_pipeline, 'xgb_final_model.pkl')

['xgb_final_model.pkl']

#### Forecasting Future Bike Demand

To generate predictions for future dates, we define a `forecast_bike_demand()` function that simulates each day recursively.

- **Historical data** (including trip counts) is combined with a date-station grid for the prediction period.
- For each forecasted date, the function:
  1. Creates features including lag and rolling statistics using the latest available data
  2. Uses the trained XGRegressor model to predict demand for every station on that day
  3. Appends the predictions back into the dataset so they can be used as lags in the next day's forecast
- This recursive structure allows the model to forecast multiple consecutive days, even when no true data exists beyond the last historical date.
- The final result is a clean DataFrame of predicted trip counts per station for the requested date range.

To support forecasting, we also provide the model with **future weather data** from **July 1, 2025 to June 30, 2026**, downloaded from the [Visual Crossing API](https://www.visualcrossing.com/). This ensures the model can incorporate temperature, precipitation, wind, and other conditions during prediction.


Below, we apply this function to predict demand from **July 1 to July 7, 2025**, using the trained model.

In [55]:
# future weather forecast from July 1 2025 through July 1 2027
weather_future = pd.read_csv('data/weather_future.csv')[['datetime','temp','humidity','precip','snow','windspeed','visibility','cloudcover']]
weather_future['date'] = pd.to_datetime(pd.to_datetime(weather_future['datetime']).dt.date)
weather_future.head()

,datetime,temp,humidity,precip,snow,windspeed,visibility,cloudcover,date
0,2025-07-01,26.6,74.0,4.220,0.0,33.2,15.5,77.6,2025-07-01
1,2025-07-02,26.1,69.2,0.013,0.0,23.8,16.0,88.1,2025-07-02
2,2025-07-03,26.3,60.0,0.014,0.0,36.6,15.9,38.9,2025-07-03
3,2025-07-04,22.6,50.3,0.000,0.0,33.0,16.0,26.4,2025-07-04
4,2025-07-05,23.8,47.0,0.000,0.0,25.5,16.0,70.3,2025-07-05


In [56]:
def forecast_bike_demand(start_date, end_date, full_data, model_pipeline):
    """
    Forecasts bike demand per station from start_date to end_date using past data and a trained pipeline.
    """

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # 1. Filter to recent historical data needed for lags
    history_window = (full_data[['start_station_id','date','trip_count']].copy())

    # 2. Get list of all active stations to forecast
    june_2025_stations = (full_data[
        (full_data['date'] >= '2025-06-01') & (full_data['date'] <= '2025-06-30')]
        ['start_station_id'].unique())
    stations = june_2025_stations

    # 3. Create empty grid of future dates × stations
    future_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    future_df = pd.DataFrame([(station, date) for station in stations for date in future_dates],
                             columns=['start_station_id', 'date'])
  

   
    # 4. Combine with historical data (to allow lag feature generation)
    combined = pd.concat([history_window, future_df], ignore_index=True, sort=False)

    # 5. Create placeholder for predictions
    all_predictions = []

    # 6. Extend forecasting window: start a bit earlier so lag_7 is computable
    recursive_start = pd.to_datetime("2025-07-01")
    forecast_dates = pd.date_range(
        start=recursive_start,
        end=end_date,
        freq='D'
    )

    # 7. Start with combined base data (historical + blank rows)
    combined = pd.concat([history_window, future_df], ignore_index=True, sort=False)
    combined = combined.merge(weather_future, on='date', how='left')

    for forecast_date in forecast_dates:
        # Create features with latest known + predicted values
        temp = create_features(combined.copy())

        # Filter to the row for this forecast date
        row = temp[temp['date'] == forecast_date]

        # Predict
        X_row = row.drop(columns=['trip_count'], errors='ignore')
        y_pred = model_pipeline.predict(X_row)

        row['trip_count'] = y_pred
        all_predictions.append(row[['date', 'start_station_id', 'trip_count']])

        # Append prediction back to combined to use in future lags
        combined = pd.concat([combined, row[['start_station_id', 'date', 'trip_count']]], ignore_index=True)

    # 8. Final result
    result_df = pd.concat(all_predictions, ignore_index=True)
    result_df.rename(columns={'trip_count': 'predicted_trip_count'}, inplace=True)
    result_df = result_df[(result_df['date'] >= start_date) & (result_df['date'] <= end_date)]

    return result_df[['date', 'start_station_id', 'predicted_trip_count']]

In [57]:
# Load saved model
loaded_model = joblib.load('xgb_final_model.pkl')

# Forecast demand for the first week of July 2025
predictions = forecast_bike_demand(
    start_date='2025-07-01',
    end_date='2025-07-07',
    full_data=daily_counts,  
    model_pipeline=loaded_model
)

predictions

/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/var/folders/cl/4ndfg5191d74xd8pclttf16w0000gn/T/ipykernel_597/2682418584.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['trip_count'] = y_pred
/Users/turkialrasheed/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/var/folders/cl/4ndfg5191d74xd8pclttf16w0000gn/T/ipykernel_597/2682418584.py:54: SettingWith

,date,start_station_id,predicted_trip_count
0,2025-07-01,A32000,19.307381
1,2025-07-01,A32001,17.419455
2,2025-07-01,A32002,39.889984
3,2025-07-01,A32003,21.561653
4,2025-07-01,A32004,41.716137
...,...,...,...
3873,2025-07-07,X32999,11.053179
3874,2025-07-07,Z32997,19.366953
3875,2025-07-07,Z32998,17.000587
3876,2025-07-07,Z32999,14.786590
